In [1]:
!nvidia-smi

Thu Jan  5 11:00:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   39C    P0    52W / 300W |      0MiB / 32508MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  Off  | 00000000:08:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2022-10-26 23:16:12.000488: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-26 23:16:12.188820: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9667993972176814861
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 32479117312
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18279729406772842858
physical_device_desc: "device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:62:00.0, compute capability: 7.0"
xla_global_id: 416903419
]


2022-10-26 23:16:15.848529: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-26 23:16:18.741713: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 30974 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:62:00.0, compute capability: 7.0


In [4]:
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
import tqdm.notebook as tqdm

In [6]:
!gdown https://drive.google.com/uc?id=1-1_OHJFk-h9O8eTg5a7lH4QFk4_LFHMG

Downloading...
From: https://drive.google.com/uc?id=1-1_OHJFk-h9O8eTg5a7lH4QFk4_LFHMG
To: /home/191it109/project/home/molecular_function.csv
100%|██████████████████████████████████████| 37.9M/37.9M [00:02<00:00, 16.8MB/s]


In [5]:
df = pd.read_csv('molecular_function.csv')
df

,Entry,Entry Name,Organism,Length,Sequence,Gene Ontology (molecular function)
0,B4ESY8,URK_PROMH,Proteus mirabilis (strain HI4320),213,MADTAHQCTIVGIAGASASGKSLIASTLYRELRAQVGDHNIGVIPE...,GO:0005524
1,Q9H479,FN3K_HUMAN,Homo sapiens (Human),309,MEQLLRAELRTATLRAFGGPGAGCISEGRAYDTDAGPVFVKVNRRT...,GO:0005524;GO:0016301
2,B4PRE2,DGKH_DROYA,Drosophila yakuba (Fruit fly),1917,MSHLKLDTLHVQRSPRGSRRSSRSSGRSSACSSGSISPVPIIPIIS...,GO:0005524;GO:0046872
3,A3DMQ4,RS4_STAMF,Staphylothermus marinus (strain ATCC 43588 / D...,168,MGDPKKPRKKWEGPRHPWRKEVLVQELKLLGTYGLRNKRELWRAQT...,GO:0019843;GO:0003735
4,P55656,Y4SO_SINFN,Sinorhizobium fredii (strain NBRC 101917 / NGR...,705,MENKSLQPPLPRSERRIRVLHNDVTIDSYGWLRDREDPDVLAYLEA...,GO:0004252
...,...,...,...,...,...,...
76736,Q7U7P8,BIOB_PARMW,Parasynechococcus marenigrum (strain WH8102),325,MTFTIRHDWTIAEIQALLELPLMELLWQAQYVHRAANPGYRVQLAS...,GO:0051537;GO:0051539;GO:0005506
76737,Q55855,GLK_SYNY3,Synechocystis sp. (strain PCC 6803 / Kazusa),355,MGAMGVNFLAGDIGGTKTILALVTINESSPGLARPVTLFEQTYSSP...,GO:0005524
76738,Q7TPM3,TRI17_MOUSE,Mus musculus (Mouse),477,MDAVELARRLQEEATCSICLDYFTDPVMTACGHNFCRECIQMSWEK...,GO:0061630;GO:0004842;GO:0008270
76739,B1IPK9,SYT_ECOLC,Escherichia coli (strain ATCC 8739 / DSM 1576 ...,642,MPVITLPDGSQRHYDHAVSPMDVALDIGPGLAKACIAGRVNGELVD...,GO:0005524;GO:0046872;GO:0000049


In [6]:
go_terms_mf = set()
for idx, row in df.iterrows():
  for term in row['Gene Ontology (molecular function)'].split(';'):
    go_terms_mf.add(term)
go_terms_mf = list(go_terms_mf)
go_terms_mf.sort()
print(len(go_terms_mf))
print(go_terms_mf)

149
['GO:0000049', 'GO:0000107', 'GO:0000166', 'GO:0000175', 'GO:0000287', 'GO:0000976', 'GO:0000977', 'GO:0000978', 'GO:0000981', 'GO:0001228', 'GO:0002161', 'GO:0003676', 'GO:0003677', 'GO:0003678', 'GO:0003682', 'GO:0003684', 'GO:0003690', 'GO:0003697', 'GO:0003700', 'GO:0003712', 'GO:0003713', 'GO:0003723', 'GO:0003724', 'GO:0003729', 'GO:0003735', 'GO:0003743', 'GO:0003746', 'GO:0003755', 'GO:0003779', 'GO:0003824', 'GO:0003861', 'GO:0003887', 'GO:0003899', 'GO:0003911', 'GO:0003924', 'GO:0003989', 'GO:0004017', 'GO:0004176', 'GO:0004190', 'GO:0004222', 'GO:0004252', 'GO:0004359', 'GO:0004372', 'GO:0004497', 'GO:0004519', 'GO:0004521', 'GO:0004523', 'GO:0004672', 'GO:0004674', 'GO:0004814', 'GO:0004826', 'GO:0004827', 'GO:0004834', 'GO:0004842', 'GO:0004930', 'GO:0005096', 'GO:0005102', 'GO:0005125', 'GO:0005179', 'GO:0005198', 'GO:0005344', 'GO:0005506', 'GO:0005507', 'GO:0005509', 'GO:0005516', 'GO:0005524', 'GO:0005525', 'GO:0008017', 'GO:0008083', 'GO:0008121', 'GO:0008137', '

In [7]:
def get_segments(sequence,segment_size=100,gap=30):
  segments = []
  start = 0
  end = segment_size
  while end <= len(sequence):
    segments.append(sequence[start:end])
    start += gap
    end += gap
  last_segment = sequence[start:]
  segments.append(last_segment)
  
  return segments

def get_training_data(df,segment_size=100,gap=30):
  training_data = list()
  for idx,row in tqdm.tqdm(df.iterrows()):
    labels = [0] * len(go_terms_mf)
    for term in row['Gene Ontology (molecular function)'].split(';'):
      labels[go_terms_mf.index(term)] = 1
    segments = get_segments(row['Sequence'],segment_size,gap)
    for segment in segments:
      training_data.append([row['Entry'],segment,labels])
  return training_data

In [8]:
training_data = get_training_data(df)
print(len(training_data))

0it [00:00, ?it/s]

872283


In [21]:
def get_ngrams(segment,n=3):
  ngrams = []
  for i in range(len(segment)-n+1):
    ngrams.append(segment[i:i+n])
  return ngrams

In [22]:
# Generate training data of ngrams
training_data_ngrams = []

for i in tqdm.tqdm(range(len(training_data))):
    training_data_ngrams.append([training_data[i][0],get_ngrams(training_data[i][1],n=3),training_data[i][2]])
print(len(training_data_ngrams))

  0%|          | 0/872283 [00:00<?, ?it/s]

872283


In [20]:
# np.save('mf/training_data_ngrams.npy',training_data_ngrams)

In [ ]:
# # Load array if already stored
# training_data_ngrams = np.load('drive/MyDrive/mp/mf/training_data_ngrams.npy',allow_pickle=True)
# print(len(training_data_ngrams))

In [100]:
def get_skip_grams(segment,skip=1,n=3):
  skip_grams = []
  window_size = skip + n
  for i in range(len(segment)-window_size+1):
    window = segment[i:i+window_size]
    indices = list(range(window_size))
    indices.pop(0)
    for idx in indices[::-1]:
      temp = ''
      for j in range(window_size):
        if j!=idx:
          temp+=window[j]
      skip_grams.append(temp)

  return skip_grams

In [111]:
# Generate training data of skip grams
training_data_skip_grams = []
for i in tqdm.tqdm(range(len(training_data))):
    training_data_skip_grams.append([training_data[i][0],get_skip_grams(training_data[i][1],n=3),training_data[i][2]])
print(len(training_data_skip_grams))

  0%|          | 0/872283 [00:00<?, ?it/s]

872283


In [75]:
# np.save('mf/training_data_skip_grams.npy',training_data_skip_grams)

In [76]:
# # Load array if already stored
# training_data_skip_grams = np.load('mf/training_data_skip_grams.npy',allow_pickle=True)
# print(len(training_data_skip_grams))

In [10]:
import tensorflow as tf
import tensorflow_addons as tfa

In [11]:
#Assuming training_data as global variable

def train_test_split(X,y,fold_no,prev_index,Kfolds=5):
    test_split = 1/Kfolds
    
    start_index = prev_index
    end_index = (fold_no + 1) * (test_split) * len(X)
    end_index = round(end_index)
    
    if end_index==len(X):
        end_index -= 1
    
    entry = training_data[end_index][0]
    entries = [sample[0] for sample in training_data]
    
    first_occurence = entries.index(entry)
    entries.reverse()
    
    last_occurence = entries.index(entry)
    last_occurence = len(entries) - last_occurence - 1
    
    del entries
    gc.collect()
    
    end_index = first_occurence if (abs(end_index-first_occurence) < abs(end_index-last_occurence)) else last_occurence
    
    X_test = X[start_index:end_index+1]
    y_test = y[start_index:end_index+1]
    X_train = X[:start_index]
    X_train.extend(X[end_index+1:])
    y_train = y[:start_index]
    y_train.extend(y[end_index+1:])
    
    return X_train, y_train, X_test, y_test, start_index, end_index + 1

In [12]:
MAX_WORDS = 13824
MAX_LEN_NG = 98 #100
MAX_LEN_SG = 291 #300

def tokenization(X_train,X_test,maxlen):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_WORDS)
    tokenizer.fit_on_texts(X_train)

    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)

    vocab_size = len(tokenizer.word_index) + 1

    X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding='post', maxlen=maxlen)
    
    return X_train, X_test, vocab_size, tokenizer

In [13]:
from tensorflow.keras import backend as K

false_negative_penalty = 6
false_positive_penalty = 1

def custom_loss(y_true, y_logit):

    loss = float(0)
    y_true = tf.cast(y_true, tf.float32)
    y_logit = tf.cast(y_logit, tf.float32)
    
    first_term = false_negative_penalty * float(y_true) * - K.log(y_logit + K.epsilon())
    second_term = false_positive_penalty * (1 - float(y_true)) * - K.log(1 - y_logit + K.epsilon())
    
    loss = K.mean(first_term+second_term)

    return loss

def precision(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=1)
    precision = true_positives / (predicted_positives + K.epsilon())
    
    return K.mean(precision)


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=1)
    recall = true_positives / (possible_positives + K.epsilon())
    return K.mean(recall)

def f1_score(y_true, y_pred):
    rec = recall(y_true,y_pred)
    prec = precision(y_true,y_pred)
    f1 = 2*prec*rec/(prec+rec)
    return f1

In [14]:
class attention(tf.keras.layers.Layer):
    
    def __init__(self, return_sequences=True,**kwargs):
        self.return_sequences = return_sequences
        super(attention,self).__init__()

    def get_config(self):
      config = super().get_config().copy()
      config.update({
          'return_sequences': self.return_sequences 
      })
      return config
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [15]:
#Base model using ngrams 
NUM_CLASSES = 149

def get_model_ng(vocab_size_ng):
    input_ngrams = tf.keras.layers.Input(shape=(MAX_LEN_NG,))

    embedding_layer = tf.keras.layers.Embedding(vocab_size_ng, 32)(input_ngrams)

    LSTM_Layer_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70,return_sequences=True))(embedding_layer)
    attention_output_1 = attention(return_sequences=False)(LSTM_Layer_1)
    dropout = tf.keras.layers.Dropout(0.3)(attention_output_1)
    dense_layer_1 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout)

    model = tf.keras.models.Model(
        inputs=input_ngrams, 
        outputs=dense_layer_1
    )
    
    return model

In [16]:
#Base model using skip grams 
NUM_CLASSES = 149

def get_model_sg(vocab_size_sg):
    input_skip_grams = tf.keras.layers.Input(shape=(MAX_LEN_SG,))

    embedding_layer = tf.keras.layers.Embedding(vocab_size_sg, 32)(input_skip_grams)

    LSTM_Layer_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70,return_sequences=True))(embedding_layer)
    attention_output_1 = attention(return_sequences=False)(LSTM_Layer_1)
    dropout = tf.keras.layers.Dropout(0.3)(attention_output_1)
    dense_layer_1 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout)

    model = tf.keras.models.Model(
        inputs=input_skip_grams, 
        outputs=dense_layer_1
    )
    
    return model

In [17]:
NUM_CLASSES = 149 #For molecular function (Change according to aspects)

def get_model_ng_sg(vocab_size_ng, vocab_size_sg):
    #Input layers

    input_ngrams = tf.keras.layers.Input(shape=(MAX_LEN_NG,)) 
    input_skip_grams = tf.keras.layers.Input(shape=(MAX_LEN_SG,)) 

    #embeddings
    embedding_layer_ngrams = tf.keras.layers.Embedding(vocab_size_ng, 32)(input_ngrams)
    embedding_layer_skip_grams = tf.keras.layers.Embedding(vocab_size_sg, 32)(input_skip_grams)

    #BI-LSTMs for each of the inputs
    sequence_output_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70,return_sequences=True))(embedding_layer_ngrams)
    attention_output_1 = attention(return_sequences=False)(sequence_output_1)
    dropout_1 = tf.keras.layers.Dropout(0.3)(attention_output_1)
    dense_layer_1 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout_1)

    sequence_output_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70, return_sequences=True))(embedding_layer_skip_grams)
    attention_output_2 = attention(return_sequences=False)(sequence_output_2)
    dropout_2 = tf.keras.layers.Dropout(0.3)(attention_output_2)
    dense_layer_2 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout_2)

    max_layer = tf.keras.layers.Maximum()([dense_layer_1,dense_layer_2])

    model = tf.keras.models.Model(
        inputs=[
            input_ngrams,
            input_skip_grams
        ], 
        outputs=max_layer)
    
    return model

In [18]:
def compute_metrics(predictions, start_index, end_index):
    final_predictions = []
    actual_y_test = []

    current_entry = ''
    counter = 0
    total_counts = 0

    if len(predictions) == len(training_data[start_index: end_index]):
        temp = np.zeros(NUM_CLASSES)
        for i in range(len(predictions)):
            if current_entry != training_data[start_index+i][0]:
                #compute prev
                if i!=0:
                    temp /= counter
                    final_predictions.append(temp)

                #reset
                total_counts += counter
                counter = 1
                temp = np.zeros(NUM_CLASSES)

                #init new
                current_entry = training_data[start_index+i][0]
                temp += np.array(predictions[i])
                actual_y_test.append(training_data[start_index+i][2])
            else:
                temp += np.array(predictions[i])
                counter += 1

        total_counts += counter
        temp /= counter
        final_predictions.append(temp)

    else:
        print('Lengths of predictions dont match with test data')
    
    final_predictions = np.array(final_predictions, dtype=float)
    actual_y_test = np.array(actual_y_test, dtype=float)
    
    rec = recall(actual_y_test,final_predictions)
    prec = precision(actual_y_test,final_predictions)
    f1 = f1_score(actual_y_test,final_predictions)
    
    return rec,prec,f1

### Model using ngrams only

In [23]:
#Considering ngrams
X_ng = [' '.join(sample[1]) for sample in training_data_ngrams]
y = [sample[2] for sample in training_data]

del training_data_ngrams

In [24]:
Kfolds = 5
prev_index = 0

recs = []
precs = []
f1s = []

for i in range(Kfolds):
    print('\n\n****Fold:', i+1,'****')
    
    print('Splitting into train-test...')
    X_train_ng, y_train, X_test_ng, y_test, start_index, prev_index = train_test_split(X_ng,y,i,prev_index,Kfolds)

    print('Tokenizing...')
    X_train_ng, X_test_ng, vocab_size_ng, tokenizer1 = tokenization(X_train_ng, X_test_ng, MAX_LEN_NG)
    
    print('Shuffling...')
    shuffled = [[X_train_ng[i],y_train[i]] for i in range(len(X_train_ng))]
    np.random.shuffle(shuffled)

    X_train_ng = [shuffled[i][0] for i in range(len(shuffled))]
    y_train = [shuffled[i][1] for i in range(len(shuffled))]
    X_train_ng = np.array(X_train_ng)
    y_train = np.array(y_train)
    
    
    
    model = get_model_ng(vocab_size_ng)
    
    model.compile(
        loss=custom_loss, 
        optimizer='adam', 
        metrics=[
            recall,
            precision,
            f1_score
        ])
    
    print('Training...')
    history = model.fit(X_train_ng, y_train, batch_size=32, epochs=5,validation_split=0.2)
    
    print('\nEvaluating model...')
    predictions = model.predict(X_test_ng)
    
    rec, prec, f1 = compute_metrics(predictions, start_index, prev_index)
    
    recs.append(rec.numpy())
    precs.append(prec.numpy())
    f1s.append(f1.numpy())

print('Recall:',sum(recs)/len(recs))
print('Precision:',sum(precs)/len(precs))
print('F1-Score:',sum(f1s)/len(f1s))



****Fold: 1 ****
Splitting into train-test...
Tokenizing...
Shuffling...
Training...
Epoch 1/5


2022-10-26 23:21:57.225729: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


17446/17446 [==============================] - 189s 11ms/step - loss: 0.1935 - recall: 0.3500 - precision: 0.2812 - f1_score: 0.3095 - val_loss: 0.1616 - val_recall: 0.4319 - val_precision: 0.3327 - val_f1_score: 0.3744
Epoch 2/5
17446/17446 [==============================] - 185s 11ms/step - loss: 0.1513 - recall: 0.4733 - precision: 0.3610 - f1_score: 0.4081 - val_loss: 0.1385 - val_recall: 0.4997 - val_precision: 0.4147 - val_f1_score: 0.4520
Epoch 3/5
17446/17446 [==============================] - 185s 11ms/step - loss: 0.1328 - recall: 0.5374 - precision: 0.4167 - f1_score: 0.4681 - val_loss: 0.1254 - val_recall: 0.5570 - val_precision: 0.4509 - val_f1_score: 0.4973
Epoch 4/5
17446/17446 [==============================] - 185s 11ms/step - loss: 0.1214 - recall: 0.5761 - precision: 0.4527 - f1_score: 0.5058 - val_loss: 0.1178 - val_recall: 0.5837 - val_precision: 0.4763 - val_f1_score: 0.5235
Epoch 5/5
17446/17446 [==============================] - 185s 11ms/step - loss: 0.1136 - r

### Model using skip grams only

In [112]:
#Considering skip grams
X_sg = [' '.join(sample[1]) for sample in training_data_skip_grams]
y = [sample[2] for sample in training_data]

del training_data_skip_grams

In [113]:
Kfolds = 5
prev_index = 0

recs = []
precs = []
f1s = []

for i in range(Kfolds):
    print('\n\n****Fold:', i+1,'****')
    
    print('Splitting into train-test...')
    X_train_sg, y_train, X_test_sg, y_test, start_index, prev_index = train_test_split(X_sg,y,i,prev_index,Kfolds)

    print('Tokenizing...')
    X_train_sg, X_test_sg, vocab_size_sg, tokenizer1 = tokenization(X_train_sg, X_test_sg, MAX_LEN_SG)
    
    print('Shuffling...')
    shuffled = [[X_train_sg[i],y_train[i]] for i in range(len(X_train_sg))]
    np.random.shuffle(shuffled)

    X_train_sg = [shuffled[i][0] for i in range(len(shuffled))]
    y_train = [shuffled[i][1] for i in range(len(shuffled))]
    X_train_sg = np.array(X_train_sg)
    y_train = np.array(y_train)
    

    model = get_model_sg(vocab_size_sg)
    
    model.compile(
        loss=custom_loss, 
        optimizer='adam', 
        metrics=[
            recall,
            precision,
            f1_score
        ])
    
    print('Training...')
    history = model.fit(X_train_sg, y_train, batch_size=32, epochs=5,validation_split=0.2)
    
    print('Evaluating model...')
    predictions = model.predict(X_test_sg)
    
    rec, prec, f1 = compute_metrics(predictions, start_index, prev_index)
    
    recs.append(rec.numpy())
    precs.append(prec.numpy())
    f1s.append(f1.numpy())

print('Recall:',sum(recs)/len(recs))
print('Precision:',sum(precs)/len(precs))
print('F1-Score:',sum(f1s)/len(f1s))



****Fold: 1 ****
Splitting into train-test...
Tokenizing...
Shuffling...
Training...
Epoch 1/5
17446/17446 [==============================] - 598s 34ms/step - loss: 0.2009 - recall: 0.3306 - precision: 0.2658 - f1_score: 0.2923 - val_loss: 0.1725 - val_recall: 0.3947 - val_precision: 0.3053 - val_f1_score: 0.3429
Epoch 2/5
17446/17446 [==============================] - 595s 34ms/step - loss: 0.1610 - recall: 0.4462 - precision: 0.3282 - f1_score: 0.3767 - val_loss: 0.1474 - val_recall: 0.4813 - val_precision: 0.3732 - val_f1_score: 0.4191
Epoch 3/5
17446/17446 [==============================] - 595s 34ms/step - loss: 0.1413 - recall: 0.5089 - precision: 0.3863 - f1_score: 0.4378 - val_loss: 0.1327 - val_recall: 0.5279 - val_precision: 0.4276 - val_f1_score: 0.4713
Epoch 4/5
17446/17446 [==============================] - 595s 34ms/step - loss: 0.1284 - recall: 0.5514 - precision: 0.4292 - f1_score: 0.4814 - val_loss: 0.1239 - val_recall: 0.5593 - val_precision: 0.4571 - val_f1_score: 

### Model using skip grams and ngrams

In [102]:
#Considering ngrams and skip grams
X_ng = [' '.join(sample[1]) for sample in training_data_ngrams]
X_sg = [' '.join(sample[1]) for sample in training_data_skip_grams]
y = [sample[2] for sample in training_data]

del training_data_ngrams
del training_data_skip_grams

In [105]:
print(len(X_ng),len(X_sg))

872283 872283


In [110]:
Kfolds = 5
prev_index = 0

recs = []
precs = []
f1s = []

for i in range(Kfolds):
    print('\n\n****Fold:', i+1,'****')
    
    print('Splitting into train-test...')
    X_train_ng, y_train, X_test_ng, y_test, start_index, prev_index1 = train_test_split(X_ng,y,i,prev_index,Kfolds)
    X_train_sg, _, X_test_sg, _, _, _= train_test_split(X_sg,y,i,prev_index,Kfolds)
    
    prev_index = prev_index1
    
    print('Tokenizing...')
    X_train_ng, X_test_ng, vocab_size_ng, tokenizer1 = tokenization(X_train_ng, X_test_ng, MAX_LEN_NG)  
    X_train_sg, X_test_sg, vocab_size_sg, tokenizer2 = tokenization(X_train_sg, X_test_sg, MAX_LEN_SG)
    
   
    print('Shuffling...')   
    shuffled = [[X_train_ng[i],X_train_sg[i],y_train[i]] for i in range(len(X_train_ng))]
    np.random.shuffle(shuffled)

    X_train_ng = [shuffled[i][0] for i in range(len(shuffled))]
    X_train_sg = [shuffled[i][1] for i in range(len(shuffled))]
    y_train = [shuffled[i][2] for i in range(len(shuffled))]
    X_train_ng = np.array(X_train_ng)
    X_train_sg = np.array(X_train_sg)
    y_train = np.array(y_train)
    
    
    model = get_model_ng_sg(vocab_size_ng, vocab_size_sg)
    
    model.compile(
        loss=custom_loss, 
        optimizer='adam', 
        metrics=[
            recall,
            precision,
            f1_score
        ])

    print('Training...')
    history = model.fit([X_train_ng,X_train_sg], y_train, batch_size=32, epochs=5,validation_split=0.2)
    
    print('Evaluating model...')
    predictions = model.predict([X_test_ng,X_test_sg])
    
    rec, prec, f1 = compute_metrics(predictions, start_index, prev_index)
    
    recs.append(rec.numpy())
    precs.append(prec.numpy())
    f1s.append(f1.numpy())

print('Recall:',sum(recs)/len(recs))
print('Precision:',sum(precs)/len(precs))
print('F1-Score:',sum(f1s)/len(f1s))



****Fold: 1 ****
Splitting into train-test...
Tokenizing...
Shuffling...
Training...
Epoch 1/5
17446/17446 [==============================] - 774s 44ms/step - loss: 0.1945 - recall: 0.3513 - precision: 0.2789 - f1_score: 0.3085 - val_loss: 0.1623 - val_recall: 0.4466 - val_precision: 0.3124 - val_f1_score: 0.3664
Epoch 2/5
17446/17446 [==============================] - 772s 44ms/step - loss: 0.1513 - recall: 0.4778 - precision: 0.3588 - f1_score: 0.4084 - val_loss: 0.1371 - val_recall: 0.5178 - val_precision: 0.4098 - val_f1_score: 0.4563
Epoch 3/5
17446/17446 [==============================] - 771s 44ms/step - loss: 0.1320 - recall: 0.5416 - precision: 0.4215 - f1_score: 0.4728 - val_loss: 0.1249 - val_recall: 0.5547 - val_precision: 0.4598 - val_f1_score: 0.5017
Epoch 4/5
17446/17446 [==============================] - 771s 44ms/step - loss: 0.1200 - recall: 0.5810 - precision: 0.4604 - f1_score: 0.5126 - val_loss: 0.1164 - val_recall: 0.5815 - val_precision: 0.4908 - val_f1_score: 